In [1]:
# Dependencies
import pandas as pd
import json
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

In [2]:
# read in happiness files for all years
happiness_file_15 = "Data/happiness_2015.csv"
happy15_df = pd.read_csv(happiness_file_15)

happiness_file_16 = "Data/happiness_2016.csv"
happy16_df = pd.read_csv(happiness_file_16)

happiness_file_17 = "Data/happiness_2017.csv"
happy17_df = pd.read_csv(happiness_file_17)

# read in region and continents data
region_cont = "Data/region_continent.csv"
region_cont_df = pd.read_csv(region_cont)


In [3]:
# Create YEAR and UNIQUE_ID fields
# 2015
happy15_df['YEAR'] = 2015
happy15_df['UNIQUE_ID'] = happy15_df['COUNTRY'] + '_' + happy15_df.YEAR.map(str)

# 2016
happy16_df['YEAR'] = 2016
happy16_df['UNIQUE_ID'] = happy16_df['COUNTRY'] + '_' + happy16_df.YEAR.map(str)

# 2017
happy17_df['YEAR'] = 2017
happy17_df['UNIQUE_ID'] = happy17_df['COUNTRY'] + '_' + happy17_df.YEAR.map(str)

In [4]:
#Merge to capture Region and Continent data
happy15_region_df = pd.merge(happy15_df, region_cont_df, on="COUNTRY")
happy16_region_df = pd.merge(happy16_df, region_cont_df, on="COUNTRY")
happy17_region_df = pd.merge(happy17_df, region_cont_df, on="COUNTRY")

In [5]:
# Set index
happy15_region_df.set_index("UNIQUE_ID", inplace=True)
happy16_region_df.set_index("UNIQUE_ID", inplace=True)
happy17_region_df.set_index("UNIQUE_ID", inplace=True)

In [6]:
# Combine all year DF's into one DF
happyall_region_df = pd.concat([happy15_region_df, happy16_region_df, happy17_region_df])

In [7]:
#Reorder fields for final output
all_cols = ['YEAR','COUNTRY','REGION','CONTINENT','HAPPINESS_RANK','HAPPINESS_SCORE','ECONOMY_GDP_PER_CAPITA',
                      'FAMILY','HEALTH_LIFE_EXPECTANCY','FREEDOM','GENEROSITY','TRUST_GOVERNMENT_CORRUPTION','DYSTOPIA_RESIDUAL']
happyall_region_df2= happyall_region_df[all_cols].copy()

In [8]:
#Create database connection
##Note: {your_password} should be inserted
connection_string = "root:datascience2018@127.0.0.1/happiness_data"
engine = create_engine(f'mysql://{connection_string}')

# confirm tables
# engine.table_names()

In [9]:
# load final df into sql database
happyall_region_df2.to_sql(name='happiness_by_region_yr', con=engine, if_exists='append', index=True)